In [1]:
"""
Created on Thur Jul  7 16:20:39 2022
Revised on Thur Jul  13 15:20:39 2022

@author: Junhyun
"""

import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
import joblib

def bootstrap_limit(stat, alpha=0.05, bootstrap=100):
    '''
        @Description
            Bootstrap sampling을 활용한 Control Limit 산출 기법

        @Parameter
            stat : 통계량 (정상상태의 데이터 입력)
            alpha : Control Limit을 정하기 위한 유의수준 (0~1)
            bootstrap : 샘플링 횟수
        @Return
            limit : 임계값 (CL : Control Limit)
    '''
    alpha = alpha*100
    alpha = 100 - alpha
    samsize = max(100, len(stat))
    
    stat = stat.reshape(len(stat)) # 2차원 array를 1차원 array로 변환
    
    # bootstrap 수 만큼 다음 작업을 반복 : samsize(최소 10000번)만큼 정상상태 데이터를 유의수준 만큼 복원 추출 후 평균 값 사용 
    limit = np.mean(list(map(lambda x:np.percentile(np.random.choice(stat,samsize,replace=True),alpha), range(0,bootstrap))))
    
    return limit

class CBM():
    """
    CBM : Clustering Base Mahalanobis distance
    """
    
    def __init__(self):
        
        self.tr_k_mu = None
        self.tr_k_cov = None
        self.cl = None
        self.n_clusters = None
        
    def fit(self, trdat, n_clusters=3, max_iter=100, alpha=0.05):
        """

        Parameters
        ----------
        trdat : array
            Train data
        n_clusters : int
            cluster의 수
        max_iter : int
            kmeans clustering 학습 횟수        
        alpha : int, 0~1
            Bootstrap Limit value. The default is 0.05.

        Returns
        -------
        trScore : array
            Train Score, 이상치 점수를 의미함. 클수록 정상패턴에서 벗어남을 의미
        CL : float
            trScore Control Limit

        """
        if isinstance(trdat,(np.ndarray)):
            trdat = pd.DataFrame(trdat)
            
        self.tr_k_mu = np.zeros((trdat.shape[1], n_clusters))
        self.tr_k_cov = np.zeros((n_clusters, trdat.shape[1], trdat.shape[1]))
        self.n_clusters = n_clusters

        km = KMeans(n_clusters=n_clusters,max_iter=max_iter,random_state=0).fit(trdat)
        km_score = km.predict(trdat)
        cluster_df = pd.concat([trdat,pd.DataFrame(km_score, columns=['cluster'])], axis=1)
        
        for i in range(n_clusters):
            self.tr_k_mu[:,i] = trdat[cluster_df['cluster'] == i].mean()
            self.tr_k_cov[i] = trdat[cluster_df['cluster'] == i].cov()

        trCbmMat = np.zeros((len(trdat), n_clusters))
        
        # train fit
        for i in range(len(trdat)):
            for j in range(n_clusters):
                # cluster 별 마할라노비스 계산
                trCbmMat[i,j] = (trdat.values[i,:] - self.tr_k_mu[:,j]) @ np.linalg.pinv(self.tr_k_cov[j]) @ (trdat.values[i,:] - self.tr_k_mu[:,j]).transpose()
        
        # CBM 이상감지 통계량
        self.trScore = trCbmMat.min(axis=1)
        self.cl = bootstrap_limit(self.trScore, alpha=alpha, bootstrap=100)
        
        return {"trScore" : self.trScore}
    
    def CL_printor(self) :
        """
        
        Returns
        -------
        CL: float
            Control Limit,
            
        """
        
        return {'CL' : self.cl}
    
    def predict(self, tsdat):
        """

        Parameters
        ----------
        tsdat : array
            Test data. 예측 대상이 되는 데이터

        Returns
        -------
        tsScore : array
            Test data의 이상치 값

        """
        if isinstance(tsdat,(np.ndarray)):
            tsdat = pd.DataFrame(tsdat)

        tsCbmMat = np.zeros((len(tsdat), self.n_clusters))     
        
        # test fit
        for i in range(len(tsdat)):
            for j in range(self.n_clusters):
                # cluster 별 마할라노비스 계산
                tsCbmMat[i,j] = (tsdat.values[i,:] - self.tr_k_mu[:,j]) @ np.linalg.pinv(self.tr_k_cov[j]) @ (tsdat.values[i,:] - self.tr_k_mu[:,j]).transpose()
    
        # CBM 이상감지 통계량
        tsScore = tsCbmMat.min(axis=1)
    
        return {"tsScore" : tsScore}   

In [2]:
def cbm(trdat, tsdat, n_clusters=3, alpha=0.05):
    """

    Parameters
    ----------
    trdat : array
        Train data. 학습 대상이 되는 데이터
    tsdat : array
        Test data. 예측 대상이 되는 데이터
    n_clusters : int
        클러스터의 개수
    alpha : float, 0~1
            Bootstrap Limit value. The default is 0.05.

    Returns
    -------
    trScore : array
        Train data의 이상치 값
    tsScore : array
        Test data의 이상치 값
    CL : float 
        Control Limit

    """
    model = CBM()
    fit = model.fit(trdat, n_clusters=n_clusters, max_iter=100, alpha=alpha)
    CL = model.CL_printor()
    pred = model.predict(tsdat)
    
     # CBM model pickle 파일로 저장
    saved_model = joblib.dump(model, 'cbm.pkl')
    
    return {'trScore':fit['trScore'], 'tsScore':pred['tsScore'], 'CL': CL['CL']}    

In [3]:
# Testing Model load
import joblib
def CBM_model_loader(pickleFile, tsdat) :
    """
    저장한 모델을 로드한 후, 로드한 모델과 데이터를 활용해 분석 결과 리턴

    Parameters
    ----------
    model : ?
        로드한 모델
    tsdat : array
        예측 데이터

    Returns
    -------
    모델 리턴과 동일

    """
    model = joblib.load(pickleFile)  
    CL = model.CL_printor()
    pred = model.predict(tsdat)

    return {'tsScore' : pred['tsScore'], 'CL' : CL['CL']}

In [4]:
df = pd.read_csv('test_data.csv', encoding='euc-kr')

trdat = df.values[0:600,:]
tsdat = df.values[600:1000, :]

cbm = cbm(trdat, tsdat, n_clusters=3, alpha=0.05)

In [5]:
CBM = CBM_model_loader('cbm.pkl', tsdat)

In [6]:
CBM['tsScore']

array([2.43162328e+01, 1.44209724e+01, 8.13357066e+00, 8.22035668e+00,
       7.54079944e+00, 1.28993780e+01, 4.11692446e+01, 7.97203346e+00,
       2.52222370e+01, 1.87709320e+03, 3.06911276e+03, 2.70377479e+02,
       1.82781482e+02, 1.22364256e+01, 2.32251580e+02, 9.46761473e+00,
       9.17893742e+00, 8.71505256e+00, 6.42783582e+00, 8.25270610e+00,
       1.28327269e+02, 1.39798522e+01, 8.24644015e+00, 7.01348037e+00,
       9.26633167e+00, 1.48276262e+01, 1.21299436e+01, 9.79506376e+04,
       1.50252448e+01, 1.32732428e+01, 1.11849215e+01, 2.33582179e+02,
       9.24030546e+00, 4.93387391e+00, 4.61704992e+04, 1.50092305e+02,
       6.71798155e+00, 2.48671820e+01, 7.19978333e+00, 2.94941206e+01,
       1.08578692e+01, 6.20217078e+02, 6.52137666e+00, 1.40259747e+01,
       2.62924960e+01, 8.37958943e+00, 1.01370775e+01, 1.35393991e+02,
       5.03177714e+01, 4.91382085e+00, 8.20364621e+00, 6.16069626e+01,
       1.10189331e+02, 2.68299411e+01, 4.96417907e+01, 1.50568667e+01,
      

In [7]:
CBM['CL']

123.58298001343019